# Pré processamento dos dados

## Imports

In [84]:
import numpy as np
import pandas as pd

## Análise superficial dos dados

In [85]:
df = pd.read_csv('../datasets/raw/data-test-analytics.csv')
df.head(5)

,id,created_at,updated_at,deleted_at,name_hash,email_hash,address_hash,birth_date,status,version,city,state,neighborhood,last_date_purchase,average_ticket,items_quantity,all_revenue,all_orders,recency,marketing_source
0,8bf7960e-3b93-468b-856e-6c6c5b56f52b,08/15/17 07:05 AM,01/14/21 11:23 AM,NaN,312d206168a318614897e8ccac43bff9,83eb3aed9a44377df80ce876dce92c9a,8b4bfaa0cbc41a16f46da15ddcd6a907,07/10/74 12:00 AM,active,2.31.7,Peixoto da Praia,AM,Aparecida 7ª Seção,01/14/21 11:23 AM,151.142942,10,906.857651,6,35,crm
1,a39535b5-4647-4680-b4f6-3aed57c1f1ff,12/31/19 09:53 PM,01/08/21 11:23 AM,NaN,de448fcb47d0d6a873b2eef52b5ee595,72678bb35e2ac84ed373e81dd9dca28c,22f1cfa1847f38da3f3cb114dd2b9247,07/06/40 12:00 AM,paused,3.30.12,Fernandes,RR,Santa Isabel,01/08/21 11:23 AM,236.991790,4,236.991790,1,41,organic_search
2,dc067cd2-c021-42bd-8c0e-beb267280e66,03/07/19 11:46 PM,01/07/21 11:23 AM,NaN,cb09e447ddc38283373d56bb46498e6a,668f4ee9add29c7bd02c485f1b7509e3,6cb47446a086ee6483b3eb954f11467a,03/18/63 12:00 AM,active,3.28.9,Lopes,RR,Estrela,01/07/21 11:23 AM,211.955597,13,2331.511572,11,42,organic_search
3,b5e4caeb-3a9b-49ed-aa33-5acd06b162c1,07/21/18 10:17 AM,01/10/21 11:23 AM,NaN,52593437a405b11b3557170680ef80c8,d3fb45188d95c8d7cc49da5b4f727c86,0a6f0c54db1e6f19347f96b50f8092a4,11/21/80 12:00 AM,active,3.34.3,Campos do Campo,PE,Confisco,01/10/21 11:23 AM,204.113227,8,1224.679359,6,39,organic_search
4,d4ff61fc-f008-4e19-b8ae-bd70cfa3ae27,06/08/18 12:09 PM,01/18/21 11:23 AM,NaN,dbda4b778a966c21904238ed2d2005db,a0f76bc49b4c43327b536da6e1a1465e,143b9f169b4fa1692f6d79b5682169b5,07/07/59 12:00 AM,active,3.19.8,das Neves,RJ,Vila Suzana Segunda Seção,01/18/21 11:23 AM,252.940997,9,2023.527980,8,31,crm


Podemos otimizar o uso de memória do dataframe, convertendo as colunas.

In [86]:
memory_before = df.memory_usage(deep=True).sum()

df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  10000 non-null  object 
 1   created_at          10000 non-null  object 
 2   updated_at          10000 non-null  object 
 3   deleted_at          505 non-null    object 
 4   name_hash           10000 non-null  object 
 5   email_hash          10000 non-null  object 
 6   address_hash        10000 non-null  object 
 7   birth_date          10000 non-null  object 
 8   status              10000 non-null  object 
 9   version             10000 non-null  object 
 10  city                10000 non-null  object 
 11  state               10000 non-null  object 
 12  neighborhood        10000 non-null  object 
 13  last_date_purchase  10000 non-null  object 
 14  average_ticket      10000 non-null  float64
 15  items_quantity      10000 non-null  int64  
 16  all_r

A partir dos metadados e olhando para os dados, podemos perceber que:
- As colunas `id`, `name_hash`, `email_hash`, `address_hash` estão relacionadas a identificação do usuário, e não são relevantes para a análise;

In [87]:
df.drop(columns=[
    'id', 'name_hash', 'email_hash', 'address_hash'
], inplace=True)

Separamos as colunas relacionadas a data, convertendo para o tipo `datetime`.

In [88]:
date_columns = ['created_at', 'updated_at', 'deleted_at', 'last_date_purchase']

for item in date_columns:
    df[item] = pd.to_datetime(df[item], format='%m/%d/%y %I:%M %p')

Encontramos um pequeno problema relacionado a data de nascimento dos usuários, onde alguns valores estão com o ano 40, o que na formatação padrão do datetime %y converteria para 2040, o que é impossível. Para resolver esse problema, utilizamos uma função para modificar a data.

In [89]:
def formatar_data(data):
    ano = int(data[6:8])
    if ano < 23:
        data = data[:6] + '20' + data[6:]
    else:
        data = data[:6] + '19' + data[6:]
        
    return data

In [90]:
df.birth_date = df.birth_date.apply(formatar_data)

In [91]:
df.birth_date = pd.to_datetime(df.birth_date, format='%m/%d/%Y %I:%M %p')

Inicialmente foi verificado que os valores `int64` poderiam ser convertidos em `int16` e `int8`.

In [92]:
for item in df.select_dtypes(include=['int64']).columns:
    aux = (df[item].astype('int8') == df[item]).unique()
    if len(aux) == 1:
        df[item] = df[item].astype('int8')
    else:
        df[item] = df[item].astype('int16')
del aux

Análogo para os valores `float64` que poderiam ser convertidos em `float32` sem perda de informação.

In [93]:
for item in df.select_dtypes(include=['float64']).columns:
    aux = np.allclose(df[item].astype('float32'), df[item])
    if aux:
        df[item] = df[item].astype('float32')
del aux

Foi analisado também a possibilidade de converter as colunas `object` para `category`, mas nem todas as colunas poderiam ter essa conversão devido ao tamanho do dataframe.

In [94]:
df.select_dtypes(include=['object']).nunique().sort_values(ascending=True)

status                 3
marketing_source       6
state                 27
neighborhood         482
city                2406
version             2905
dtype: int64

In [95]:
to_cat = df.select_dtypes(include=['object']).nunique().sort_values(ascending=True)[:3].index

for item in to_cat:
    df[item] = df[item].astype('category')

In [96]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   created_at          10000 non-null  datetime64[ns]
 1   updated_at          10000 non-null  datetime64[ns]
 2   deleted_at          505 non-null    datetime64[ns]
 3   birth_date          10000 non-null  datetime64[ns]
 4   status              10000 non-null  category      
 5   version             10000 non-null  object        
 6   city                10000 non-null  object        
 7   state               10000 non-null  category      
 8   neighborhood        10000 non-null  object        
 9   last_date_purchase  10000 non-null  datetime64[ns]
 10  average_ticket      10000 non-null  float32       
 11  items_quantity      10000 non-null  int8          
 12  all_revenue         10000 non-null  float32       
 13  all_orders          10000 non-null  int8       

In [97]:
memory_after = df.memory_usage(deep=True).sum()

In [98]:
def bytes_to_megabytes(num):
    return round(num / 1024**2, 2)

memory_before, memory_after = bytes_to_megabytes(memory_before), bytes_to_megabytes(memory_after)
del bytes_to_megabytes

Por fim temos um grande ganho de memória.

In [99]:
print(f'Antes do tratamento: {memory_before} mb')
print(f'Depois do tratamento: {memory_after} mb')
print(f'Diferença bruta: {memory_before - memory_after} mb')
print(f'Diferença percentual: {(memory_before - memory_after) / memory_before * 100:.2f}%')

Antes do tratamento: 10.81 mb
Depois do tratamento: 2.64 mb
Diferença bruta: 8.17 mb
Diferença percentual: 75.58%


E por fim, salvamos o dataframe em um arquivo `pickle` para otimizar o carregamento dos dados.

In [100]:
df.to_pickle('../datasets/processed/data-test-analytics.pkl')